In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
solve_u_data = Path.home().joinpath("workspace", "pyplier", "tests", "data", "solve_u")

In [4]:
z_file = solve_u_data / "z.csv.gz"
z = pd.read_csv(z_file, index_col="gene")

In [5]:
chat_file = solve_u_data / "chat.csv.gz"
chat = pd.read_csv(chat_file, index_col="pathway")

In [6]:
pm_file = solve_u_data / "prior_mat.csv.gz"
prior_mat = pd.read_csv(pm_file, index_col="gene")
prior_mat.columns.name = "pathway"

In [7]:
penalty_factor = np.loadtxt( solve_u_data / "penalty_factor.csv.gz")

In [8]:
u_file_complete = solve_u_data / "u_complete.csv.gz"
u_complete = pd.read_csv(u_file_complete, index_col="pathway")

In [9]:
u_complete.columns = np.subtract(u_complete.columns.str.replace("V", "").astype(int), 1)

In [10]:
u_complete = u_complete.astype(np.float64)

In [11]:
u_file_fast = solve_u_data / "u_fast.csv.gz"
u_fast = pd.read_csv(u_file_fast, index_col="pathway")
u_fast.columns = np.subtract(u_fast.columns.str.replace("V", "").astype(int), 1).astype(object)
u_fast = u_fast.astype(np.float64)

In [12]:
ur = chat @ z  # get U by OLS

In [13]:
ur = ur.rank(axis="index", ascending=False)  # rank

In [14]:
iip = np.where([ur.min(axis=1) <= 10])[1]

In [15]:
results = {}

In [16]:
u = np.zeros(shape=(prior_mat.shape[1], z.shape[1]))

In [17]:
lambdas = np.exp(np.arange(start=-4, stop=-12.125, step=-0.125))

In [18]:
l_mat = np.full((len(lambdas), z.shape[1]), np.nan)

In [244]:
from glmnet import ElasticNet

In [245]:
gres = ElasticNet(
    alpha=0.9,
    lower_limits=0,
    lambda_path=lambdas,
    fit_intercept=True,
    standardize=False,
    random_state=0
    # max_features=150,
)
...
gres.fit(
    X=prior_mat.iloc[:, iip],
    y=z.iloc[:, i],
    relative_penalties=penalty_factor[iip],
)

ElasticNet(alpha=0.9,
           lambda_path=array([1.83156389e-02, 1.61634946e-02, 1.42642339e-02, 1.25881422e-02,
       1.11089965e-02, 9.80365504e-03, 8.65169520e-03, 7.63509422e-03,
       6.73794700e-03, 5.94621736e-03, 5.24751840e-03, 4.63091873e-03,
       4.08677144e-03, 3.60656314e-03, 3.18278080e-03, 2.80879419e-03,
       2.47875218e-03, 2.18749112e-03, 1.93045414e-03, 1.70361980e-03,
       1.50343919...
       7.48518299e-05, 6.60565080e-05, 5.82946637e-05, 5.14448602e-05,
       4.53999298e-05, 4.00652974e-05, 3.53575009e-05, 3.12028850e-05,
       2.75364493e-05, 2.43008313e-05, 2.14454083e-05, 1.89255064e-05,
       1.67017008e-05, 1.47391992e-05, 1.30072977e-05, 1.14788999e-05,
       1.01300936e-05, 8.93977622e-06, 7.88932483e-06, 6.96230472e-06,
       6.14421235e-06]),
           lower_limits=0, random_state=0, standardize=False)

```{R}
glmnet(
    y               = Z[,i],
    x               = priorMat[,iip],
    penalty.factor  = penalty.factor[iip],
    alpha           = glm_alpha,
    lower.limits    = 0,
    lambda          = lambdas,
    intercept       = TRUE,
    standardize     = FALSE
)
```

In [22]:
gres

ElasticNet(alpha=0.9,
           lambda_path=array([1.83156389e-02, 1.61634946e-02, 1.42642339e-02, 1.25881422e-02,
       1.11089965e-02, 9.80365504e-03, 8.65169520e-03, 7.63509422e-03,
       6.73794700e-03, 5.94621736e-03, 5.24751840e-03, 4.63091873e-03,
       4.08677144e-03, 3.60656314e-03, 3.18278080e-03, 2.80879419e-03,
       2.47875218e-03, 2.18749112e-03, 1.93045414e-03, 1.70361980e-03,
       1.50343919...
       7.48518299e-05, 6.60565080e-05, 5.82946637e-05, 5.14448602e-05,
       4.53999298e-05, 4.00652974e-05, 3.53575009e-05, 3.12028850e-05,
       2.75364493e-05, 2.43008313e-05, 2.14454083e-05, 1.89255064e-05,
       1.67017008e-05, 1.47391992e-05, 1.30072977e-05, 1.14788999e-05,
       1.01300936e-05, 8.93977622e-06, 7.88932483e-06, 6.96230472e-06,
       6.14421235e-06]),
           lower_limits=0, max_features=150, standardize=False)

In [35]:
z.shape[1]

30

In [38]:
max_path=10

In [36]:
i = 0

In [39]:
iip = np.where([ur.iloc[:, i] <= max_path])[1]

In [242]:
gres.fit(X=prior_mat.iloc[:,iip], y=z.iloc[:,i], relative_penalties=penalty_factor[iip])

TypeError: ElasticNet.fit() got an unexpected keyword argument 'random_state'

In [26]:
import glum

In [64]:
z.iloc[:,i].shape

(5892,)

In [65]:
prior_mat.iloc[:,iip].shape

(5892, 10)

In [89]:
len(penalty_factor[iip])

10

In [373]:
from sklearn.preprocessing import StandardScaler

In [377]:
StandardScaler().fit_transform(prior_mat.iloc[:,iip])

array([[-0.06910063, -0.07389689, -0.19694422, ..., -0.0583609 ,
        -0.0583609 , -0.07389689],
       [-0.06910063, -0.07389689, -0.19694422, ..., -0.0583609 ,
        -0.0583609 , -0.07389689],
       [-0.06910063, -0.07389689, -0.19694422, ..., -0.0583609 ,
        -0.0583609 , -0.07389689],
       ...,
       [-0.06910063, -0.07389689, -0.19694422, ..., -0.0583609 ,
        -0.0583609 , -0.07389689],
       [-0.06910063, -0.07389689, -0.19694422, ..., -0.0583609 ,
        -0.0583609 , -0.07389689],
       [-0.06910063, -0.07389689, -0.19694422, ..., -0.0583609 ,
        -0.0583609 , -0.07389689]])

In [394]:
glm = glum.GeneralizedLinearRegressor(
    l1_ratio=0.9,
    alpha=lambdas,
    alpha_search=True,
    fit_intercept=True,
    scale_predictors=False,
    family="gaussian",
    # link="log",
    solver="irls-cd",
    lower_bounds=0,
    random_state=0,
)

glm.fit(
    X=np.add(prior_mat.iloc[:,iip], penalty_factor[iip]),
    y=z.iloc[:,i],
)

GeneralizedLinearRegressor(alpha=array([1.83156389e-02, 1.61634946e-02, 1.42642339e-02, 1.25881422e-02,
       1.11089965e-02, 9.80365504e-03, 8.65169520e-03, 7.63509422e-03,
       6.73794700e-03, 5.94621736e-03, 5.24751840e-03, 4.63091873e-03,
       4.08677144e-03, 3.60656314e-03, 3.18278080e-03, 2.80879419e-03,
       2.47875218e-03, 2.18749112e-03, 1.93045414e-03, 1.70361980e-03,
       1.50343919...
       4.53999298e-05, 4.00652974e-05, 3.53575009e-05, 3.12028850e-05,
       2.75364493e-05, 2.43008313e-05, 2.14454083e-05, 1.89255064e-05,
       1.67017008e-05, 1.47391992e-05, 1.30072977e-05, 1.14788999e-05,
       1.01300936e-05, 8.93977622e-06, 7.88932483e-06, 6.96230472e-06,
       6.14421235e-06]),
                           alpha_search=True, family='gaussian', l1_ratio=0.9,
                           lower_bounds=0, random_state=0, solver='irls-cd')

In [395]:
glm.coef_

array([0.00203855, 0.        , 0.00283459, 0.00185107, 0.00601009,
       0.00422477, 0.00147429, 0.00152942, 0.00101545, 0.00170945])

In [396]:
glm.coef_ - gres.coef_

array([0.00066326, 0.        , 0.00011246, 0.00060242, 0.00092998,
       0.00057428, 0.00042962, 0.0008938 , 0.00082359, 0.00053135])

In [390]:
glm.coef_ - gres.coef_

array([0.00066326, 0.        , 0.00011246, 0.00060242, 0.00092998,
       0.00057428, 0.00042962, 0.0008938 , 0.00082359, 0.00053135])

In [386]:
(np.sum(np.where(glm.coef_path_ > 0, 1, 0), axis=1))

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  3,  3,  3,  3,
        3,  6,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])

In [387]:
(np.sum(np.where(glm.coef_path_ > 0, 1, 0), axis=1)) == np.sum(np.where(gres.coef_path_ > 0, 1, 0), axis=0)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [300]:
gres.coef_

array([0.0013753 , 0.        , 0.00272214, 0.00124865, 0.00508011,
       0.0036505 , 0.00104467, 0.00063562, 0.00019187, 0.00117809])

In [301]:
np.sum(np.where(gres.coef_path_ > 0, 1, 0), axis=0)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 7, 7, 7, 8, 9, 9, 9, 9, 9])

In [224]:
from sklearn import linear_model as lm

In [234]:
sens = lm.ElasticNet(
    alpha=lambdas,
    fit_intercept=True,
    positive=False,
    random_state=None,
)

In [235]:
sen.fit(X=prior_mat.iloc[:,iip], y=z.iloc[:,i],)

ElasticNet(alpha=0.01831563888873418)

In [241]:
sen.n_features_in_

10

In [231]:
(np.sum(np.where(sen.coef_ > 0, 1, 0), axis=1))

AxisError: axis 1 is out of bounds for array of dimension 1